# Business Ownership Trends in the United States

## Foundational Questions: 
1. How is the minority representation in business ownership throughout industries changing?
2. Are there any states that have higher than average minority business ownership across all classes?
3. What other ways can businesses be clustered besides sector, and what is the minority breakdown across these clusters?


For this project I wil be examining the trends in business ownership in the United States. The data for this project was acquired through the United States Census Bureau's Survey of Business Owners. The Survey of Business Owners is conducted every 5 years. The Census Bureau publishes the summary statistics for each survey based on traits like race, ethnicity, education level, and whether a person was born in the United States. I will begin the project looking at trends in this data for the survey years 2002, 2007, and 2012. This data can be found here: https://www.census.gov/programs-surveys/sbo/data/tables.html

The Census Bureau has also releases a microdata sample for the year 2007. This sample contains data regarding the characteristics of individual businesses and their respective owners for about 1.5 million businesses. To protect the identity of the businesses and their owners artificial noise is inserted into the dataset. Notably, this dataset also contains geographic information about each business as well as estimated for their employees, payroll, and receivables. This dataset can be found here: https://www.census.gov/content/census/en/data/datasets/2007/econ/sbo/2007-sbo-pums.html

In [ ]:
import requests
import pandas as pd
import numpy as np
import zipfile
import wget
import os
import glob
import warnings
warnings.filterwarnings('ignore')

import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import chart_studio.plotly as py
import seaborn as sns

from kmodes.kprototypes import KPrototypes
from sklearn import preprocessing
import time

## 1) Reading in the Data
The government API for the Survey of Business Owners is still in its fledgling stages. As of now the only year that is available in the API. Below is an example of what the API call would look like. However, for consistency purposes, I scraped the .dat files for each of the three years from the website directly rather that going through the API. 

In [ ]:
api_key='c8911a06059ba719933e4ed122c676f87fbf0b8a'

response=requests.get('https://api.census.gov/data/2012/sbo/cscb?get=FIRMALL,FIRMPDEMP_PCT_S,RCPALL,FIRMNOPD_S,RCPNOPD,FIRMNOPD_PCT,FIRMNOPD,EMP_S,PAYANN,RCPPDEMP,RCPPDEMP_F,RCPNOPD_PCT_S,FIRMALL_S,SPOUSES,CBGROUP_TTL,EMP,RCPPDEMP_S,EMP_PCT,YEAR,RCPNOPD_PCT,FIRMNOPD_PCT_S,EMP_PCT_S,CBGROUP,NAICS2012_TTL,PAYANN_PCT_S,PAYANN_S,GEO_TTL,RCPALL_PCT,RCPPDEMP_PCT_S,PAYANN_PCT,SPOUSES_TTL,RCPALL_PCT_S,RCPNOPD_S,FIRMPDEMP_PCT,FIRMPDEMP_S,FIRMPDEMP,FIRMALL_PCT,FIRMALL_PCT_S,RCPPDEMP_PCT,COUNTY&for=us:*&key=c8911a06059ba719933e4ed122c676f87fbf0b8a')
print(response.text[:1000])

### 1a) Reading DAT files from the Census Bureau website
The process had three parts: iterate through each of the files, download the zip files, then unzip them

In [ ]:
def get_zips():
    """Downloads all the the table zip files for the years 2002, 2007 and 2012"""
    base_url = r'https://www2.census.gov/econ{0}/SB/sector00/SB{1}00CSCBO0{2}.zip'
    for i in range(2002, 2013, 5):
        year = str(i)
        for j in range(1,10): #Each year had nine files numbered as such
            file_num = str(j)
            updated_url = base_url.format(year, year[-2:], file_num)
            #path to save the files my filesystem.
            path = r'./data/zips/{0}/SB{1}00CSCBO0{2}.zip'.format(year, year[-2:], file_num)
            
            download_url(updated_url, path)
            
            
def download_url(url, save_path, chunk_size=128):
    """Function to download a zip file from a url and save it to a local folder"""
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd: 
        #Files are too large to download directly with wget, so we downoad them in chunks
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)
                    
def unzip():
    """Unzips all the files in the data folders"""
    file_names = glob.glob("./data/*/*/*.zip")
    for file in file_names:
        try:
            with zipfile.ZipFile(file, 'r') as zip_ref:
                zip_ref.extractall(file[0:7] + 'unzipped' + file[11:17])
        except:
            continue
        

get_zips()
unzip()           

### 1b) Combining the files and saving them into a dataframe for each year 
Combining the data for each year in a way that made sense for analyzing trends was a considerable challenge. The structure of the data for each of the three years were different. Specifically, all the columns had different names and for some years multiple columns were combined. 

In [ ]:
def combine_tables_by_year(year_str, valid_cols, secondary_cols):
    year_df = pd.DataFrame()
    
    file_names = glob.glob("./data/unzipped/{}/*.dat".format(year_str))
        
    for file in file_names:
        #opens each file and reads it into a dataframe. Data is separated using a "|"
        infile = open(file, 'r')
        data = []
        for line in infile:
            data.append(line.split('|'))
                
        rows = data[1:]
        cols = data[0]
        dataframe = pd.DataFrame(rows, columns=cols)
            
        #changes the structure of the dataframe so the feature column will be split into two columns, the 
        #column name and the column value. This is done so that many different types of tables with different 
        #variables can be combined into one dataframe. 
        df_cols = []
        df_col = ""
        for col in dataframe.columns:
            if col in secondary_cols:
                df_cols = valid_cols + [col]
                df_col = col
                break

        dataframe["Column Val"] = dataframe[col]
        dataframe["Col Type"] = df_col
        
            
        #Final step: concatenates the individual dataframe read from the file into the master dataframe
        var_df = dataframe[valid_cols + ["Column Val"] + ["Col Type"]]
        year_df = pd.concat([year_df, var_df])
            
    return year_df


#Defining column structure for each year
valid_cols_2002 = ['NAICS2002_MEANING','YEAR', 'OWNALL', 'OWNALL_PCT']
secondary_cols_2002 = ['PRMINC_MEANING', 'PFNCT_MEANING', 'VETSPECIFIC_MEANING', 'OWNRAGE_MEANING',
                 'YRACQBUS_MEANING', 'HRSWRKD_MEANING', 'ACQBUS_MEANING', 'EDUC_MEANING', 'OWNER_SEX_GROUP_MEANING']

valid_cols_2007 = ['NAICS2007_MEANING', 'CBOGROUP_MEANING', 'CBOSEX_MEANING',
              'YEAR', 'OWNALL', 'OWNALL_PCT']
secondary_cols_2007 = ['PRMINC_MEANING', 'PFNCT_MEANING', 'VETSPECIFIC_MEANING', 'OWNRAGE_MEANING',
                 'YRACQBUS_MEANING', 'HRSWRKD_MEANING', 'ACQBUS_MEANING', 'EDUC_MEANING']

valid_cols_2012 = ['NAICS2012_TTL', 'CBOGROUP_TTL', 'CBOSEX_TTL','YEAR', 'OWNALL', 'OWNALL_PCT']
secondary_cols_2012 = ['PRMINC_TTL', 'PFNCT_TTL', 'VETSPECIFIC_TTL', 'OWNRAGE_TTL',
                 'YRACQBUS_TTL', 'HRSWRKD_TTL', 'ACQBUS_TTL', 'EDUC_TTL']

#creating dataframes for each year
df_2002 = combine_tables_by_year('2002', valid_cols_2002, secondary_cols_2002)
df_2002['OWNALL'] = pd.to_numeric(df_2002['OWNALL'])
df_2007 = combine_tables_by_year('2007', valid_cols_2007, secondary_cols_2007)
df_2007['OWNALL'] = pd.to_numeric(df_2007['OWNALL'])
df_2012 = combine_tables_by_year('2012', valid_cols_2012, secondary_cols_2012)
df_2012['OWNALL'] = pd.to_numeric(df_2012['OWNALL'])


In [ ]:
df_2002.shape

In [ ]:
df_2007.shape

In [ ]:
df_2012.shape

### 1c) The  2007 Microdata sample
Unfortunately, the Census Bureau does not provide the characteristic dart for each of the roughly 70 million businesses in the United States for each year the survey of business owners is conducted. Rather, they make available to the public one "microdata" sample of 1.5 million businesses from the year 2007. This is the most granular dataset they provide, as it contains detailed information about each business's size, performance, and ownership. It also gives state level data, a useful tool when looking for geographic differences in business ownership trends. 

In [ ]:
full_2007_df = pd.read_csv('./data/pums.csv')
full_2007_df.head()
print(len(full_2007_df.columns))

In [ ]:
#selecting the necessary columns
full_df = full_2007_df[['FIPST', 'SECTOR','EMPLOYMENT_NOISY','PAYROLL_NOISY' ,'RECEIPTS_NOISY' ,'PCT1' ,'ETH1' ,
                        'RACE1' ,'SEX1' ,'VET1','FOUNDED1' ,'PURCHASED1' ,'INHERITED1', 'RECEIVED1', 'ACQUIRENR1',
                        'ACQYR1', 'PROVIDE1', 'MANAGE1', 'FINANCIAL1', 'FNCTNABV1', 'FNCTNR1','HOURS1', 'PRMINC1', 
                        'SELFEMP1', 'EDUC1', 'AGE1', 'BORNUS1', 'DISVET1', 'ESTABLISHED', 'SCAMOUNT', 'FEDERAL',
                        'STATELOCAL', 'INDIVIDUALS', 'FAMILYBUS', 'ECOMMPCT', 'HUSBWIFE', 'HOMEBASED', 'FRANCHISE']]

analy_df = full_df[full_df['PCT1'] > 50]
analy_df.shape

## 2) Cleaning the Data

### Cleaning the 2007 PUMS data.

In [ ]:
analysis_df = analy_df.copy()


### Mapping catergorical data for all the columns
sector_map = {11:'Agriculture, Forestry, Fishing and Hunting', 21:'Mining', 22:'Utilities', 23:'Construction',
              31:'Manufacturing', 32:'Manufacturing', 33:'Manufacturing', 42:'Wholesale Trade', 44:'Retail Trade',
              45:'Retail Trade', 48:'Transportation and Warehousing', 49:'Transportation and Warehousing', 51:'Information',
              52:'Finance and Insurance', 53:'Real Estate Rental and Leasing', 54:'Professional, Scientific, and Technical Services',
              55:'Management of Companies and Enterprises', 56:'Administrative and Support and Waste Management and Remediation Services',
              61:'Educational Services', 62:'Health Care and Social Assistance', 71:'Arts, Entertainment, and Recreation', 
              72:'Arts, Entertainment, and Recreation', 81:'Other Services', 92:'Public Administration'}

general_map = {0: "Not Reported", 1: 'Yes', 2:'No'}

education_map = {0: "Not Reported", 1: 'Less than High School', 2:'High School', 3:'Technical School',
                 4:'Some College',5:'Associates',6: 'Bachelors', 7:'Grad School'}

analysis_df['FAMILYBUS'] = analysis_df['FAMILYBUS'].map(general_map)
analysis_df['BORNUS1'] = analysis_df['BORNUS1'].map(general_map)
analysis_df['VET1'] = analysis_df['VET1'].map(general_map)
analysis_df['HOMEBASED'] = analysis_df['HOMEBASED'].map(general_map)
analysis_df['FRANCHISE'] = analysis_df['FRANCHISE'].map(general_map)
analysis_df['EDUC1'] = analysis_df['EDUC1'].map(education_map)



###REformatting the State and Race columns so they are more understadable.
states = ["AL", "AK", "American Samoa", "AZ", "AR", "CA", "", "CO", "CT", "DC", "DE", "FL", "GA", 
          "Guam", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", 'Puerto Rico', "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "Virgin Islands", "WA", "WV", "WI", "WY"]

state_vals = [float(i) for i in range(1, 57)]

state_map = dict(zip(state_vals, states))

analysis_df['RACE ETH'] = analysis_df['RACE1'] + analysis_df['ETH1']

def race_eth(string):
    if string[-1] == 'H':
        return "Hispanic"
    elif string[0] == 'W':
        return "White"
    elif string[0] == 'A':
        return "Asian"
    elif string[0] == 'B':
        return "Black or African American"
    elif string[0] == 'I':
        return "American Indian or Alaska Native"
    elif string[0] == 'P':
        return "Pacific Islander"
    
analysis_df['RACE ETH'] = analysis_df['RACE ETH'].apply(race_eth)


state_num_list = [str(i) for i in [1, 4, 5, 6, 8, 9, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25,
       26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 45, 47,
       48, 49, 51, 53, 54, 55, '01', '04', '05', '06', '08', '09', '12',
       '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35',
       '36', '37', '39', '40', '41', '42', '45', '47', '48', '49', '51',
       '53', '54', '55']]

analysis_df['FIPST'] = analysis_df[analysis_df['FIPST'].isin(state_num_list)]['FIPST']

analysis_df['FIPST'] = analysis_df[analysis_df['FIPST'].notnull()]['FIPST']
analysis_df['FIPST'] = pd.to_numeric(analysis_df['FIPST'])

analysis_df['STATE'] = analysis_df['FIPST'].map(state_map)

### Creating a minority column
def is_minority(data):
    if data == 'White':
        return 'No'
    else:
        return 'Yes'
    
analysis_df['Minority'] = analysis_df['RACE ETH'].apply(is_minority)

analysis_df['SECTOR'] = analysis_df['SECTOR'].astype('category')
analysis_df[['ESTABLISHED', 'SCAMOUNT', 'FEDERAL','STATELOCAL', 'INDIVIDUALS', 'FAMILYBUS', 'ECOMMPCT', 'HUSBWIFE', 'HOMEBASED', 'FRANCHISE']] = \
analysis_df[['ESTABLISHED', 'SCAMOUNT', 'FEDERAL','STATELOCAL', 'INDIVIDUALS', 'FAMILYBUS', 'ECOMMPCT', 'HUSBWIFE', 'HOMEBASED', 'FRANCHISE']].astype('category')

### Cleaning the Yearly Data

In [ ]:
df_2002.head()

In [ ]:
#creating a dataframe that sums business owner statistics by sector
bysector_2002 = df_2002[(df_2002['Column Val']=='All owners of respondent firms') & 
                        (df_2002['OWNALL']!='0') & 
                        (df_2002['NAICS2002_MEANING']!='Total for all sectors')]

bysector_2002 = bysector_2002.drop_duplicates(subset=['NAICS2002_MEANING', 'Column Val'], keep='first')

In [ ]:
#creating a dataframe that sums business owner statistics by sector
bysector_2007 = df_2007[(df_2007['Column Val']=='All owners of respondent firms') & 
                        (df_2007['OWNALL']!='0') & (df_2007['NAICS2007_MEANING']!='Total for all sectors')
                        & (df_2007['CBOGROUP_MEANING']=='All owners of respondent firms') & 
                        (df_2007['CBOSEX_MEANING']=='All owners of respondent firms')]

bysector_2007 = bysector_2007.drop_duplicates(subset=['NAICS2007_MEANING', 'Column Val'], keep='first')

In [ ]:
#creating a dataframe that sums business owner statistics by sector
bysector_2012 = df_2012[(df_2012['Column Val']=='Total reporting') & (df_2012['OWNALL']!='0') & 
                        (df_2012['NAICS2012_TTL']!='Total for all sectors') & 
                        (df_2012['CBOGROUP_TTL']=='All owners of respondent firms') & 
                        (df_2012['CBOSEX_TTL']=='All owners of respondent firms')]

bysector_2012 = bysector_2012.drop_duplicates(subset=['NAICS2012_TTL', 'Column Val'], keep='first')

In [ ]:
#Makes the sectors consistent across the datasets
sector_map_2002 = dict(zip(list(bysector_2002['NAICS2002_MEANING'].unique()),
                           list(bysector_2012['NAICS2012_TTL'].unique()) ))

bysector_2002['NAICS2002_MEANING'] = bysector_2002['NAICS2002_MEANING'].map(sector_map_2002)
df_2002['NAICS2002_MEANING'] = df_2002['NAICS2002_MEANING'].map(sector_map_2002)

In [ ]:
bysector_2012 = bysector_2012[['NAICS2012_TTL', 'YEAR','OWNALL', 'OWNALL_PCT', 'Column Val', 'Col Type']]
bysector_2007 = bysector_2007[['NAICS2007_MEANING', 'YEAR','OWNALL', 'OWNALL_PCT', 'Column Val', 'Col Type']]

bysector_2002.columns = ['NAICS_MEANING', 'YEAR', 'OWNALL', 'OWNALL_PCT', 'Column Val','Col Type']
bysector_2007.columns = ['NAICS_MEANING', 'YEAR', 'OWNALL', 'OWNALL_PCT', 'Column Val','Col Type']
bysector_2012.columns = ['NAICS_MEANING', 'YEAR', 'OWNALL', 'OWNALL_PCT', 'Column Val','Col Type']

#combining the sector data for each of the three years
bysector_combined = pd.concat([bysector_2002, bysector_2007, bysector_2012])
bysector_combined.head()

In [ ]:
#Splitting the datasets by minority and non-minorty
df_2002_nomin = df_2002[df_2002['Column Val'] == 'White owners of respondent firms']
df_2002_min = df_2002[(df_2002['Column Val'] == 'Asian owners of respondent firms') | (df_2002['Column Val'] == 'Asian owners of respondent firms') | (df_2002['Column Val'] == 'Black or African American owners of respondent firms') | (df_2002['Column Val'] == 'Hispanic owners of respondent firms') | (df_2002['Column Val'] == 'Native Hawaiian and Other Pacific Islander owners of respondent firms')]
df_2002_nomin['CBOGROUP_MEANING'] = "Nonminority"
df_2002_min['CBOGROUP_MEANING'] = "Minority"

#total of minorities for each sector in 2012
byrace_2012 = df_2012[(df_2012['Column Val']=='Total reporting') & (df_2012['OWNALL']!=0) & ((df_2012['CBOGROUP_TTL']=='Minority') | (df_2012['CBOGROUP_TTL']=='Nonminority')) & (df_2012['CBOSEX_TTL']=='All owners of respondent firms')]
byrace_2012 = byrace_2012.drop_duplicates(subset=['NAICS2012_TTL', 'CBOGROUP_TTL', 'Column Val'], keep='first')

#total of minorities for each sector in 2007
byrace_2007 = df_2007[(df_2007['Column Val']=='All owners of respondent firms') & (df_2007['OWNALL']!=0) & ((df_2007['CBOGROUP_MEANING']=='Minority') | (df_2007['CBOGROUP_MEANING']=='Nonminority')) & (df_2007['CBOSEX_MEANING']=='All owners of respondent firms')]
byrace_2007 = byrace_2007.drop_duplicates(subset=['NAICS2007_MEANING', 'CBOGROUP_MEANING', 'Column Val'], keep='first')

#total of minorities for each sector in 2002
byrace_2002_nomin = df_2002_nomin[(df_2002_nomin['OWNALL']!='0')]
byrace_2002_nomin = byrace_2002_nomin.drop_duplicates(subset=['NAICS2002_MEANING', 'Column Val'], keep='first')
byrace_2002_nomin = byrace_2002_nomin.groupby("NAICS2002_MEANING", as_index=False).agg({'YEAR':'first', 'OWNALL':'sum', 'OWNALL_PCT': 'first', 'Column Val':'first', 'Col Type':'first', 'CBOGROUP_MEANING':'first'})
byrace_2002_min = df_2002_min[(df_2002_min['OWNALL']!='0')]
byrace_2002_min = byrace_2002_min.drop_duplicates(subset=['NAICS2002_MEANING', 'Column Val'], keep='first')
byrace_2002_min = byrace_2002_min.groupby("NAICS2002_MEANING", as_index=False).agg({'YEAR':'first', 'OWNALL':'sum', 'OWNALL_PCT': 'first', 'Column Val':'first', 'Col Type':'first', 'CBOGROUP_MEANING':'first'})

#Standardizing the columns for each of the four dataframes
byrace_2012 = byrace_2012[['NAICS2012_TTL', 'YEAR','OWNALL', 'OWNALL_PCT', 'Column Val', 'Col Type', 'CBOGROUP_TTL']]
byrace_2007 = byrace_2007[['NAICS2007_MEANING', 'YEAR','OWNALL', 'OWNALL_PCT', 'Column Val', 'Col Type', 'CBOGROUP_MEANING']]

byrace_2002_nomin.columns = ['NAICS_MEANING', 'YEAR', 'OWNALL', 'OWNALL_PCT', 'Column Val','Col Type', 'CBOGROUP_MEANING']
byrace_2002_min.columns = ['NAICS_MEANING', 'YEAR', 'OWNALL', 'OWNALL_PCT', 'Column Val','Col Type', 'CBOGROUP_MEANING']
byrace_2007.columns = ['NAICS_MEANING', 'YEAR', 'OWNALL', 'OWNALL_PCT', 'Column Val','Col Type', 'CBOGROUP_MEANING']
byrace_2012.columns = ['NAICS_MEANING', 'YEAR', 'OWNALL', 'OWNALL_PCT', 'Column Val','Col Type', 'CBOGROUP_MEANING']

#Concatenating all four of the datasets
byrace_combined = pd.concat([byrace_2002_nomin, byrace_2002_min, byrace_2007, byrace_2012])
byrace_combined.head()

## 3) Visualization
#### We start with a high level overview of all the data. To get an idea of overall trends we will look at the number of business owners in each sector for each year. 

In [ ]:
#list of individual sector
sectors = list(bysector_combined['NAICS_MEANING'].unique())

fig = go.Figure(layout = go.Layout(
    plot_bgcolor='rgba(0,0,0,0)'
))

#plots a line chart for the 3-year trend in each sector.
for sector in sectors:
    fig.add_trace(go.Scatter(x=pd.to_numeric(bysector_combined[bysector_combined['NAICS_MEANING'] == sector]['YEAR']),
                             y=pd.to_numeric(bysector_combined[bysector_combined['NAICS_MEANING'] == sector]['OWNALL']), 
                             mode="lines+markers", name=sector, visible=False,
                             marker=dict(
                                size=16,
                                color=np.random.randn(500), #set color equal to a variable
                                colorscale='Viridis', # one of plotly colorscales
                                showscale=False
                             ),
                             line=dict(color='green', width=4),
))

#logic for the buttons in the dropdown menus. Only the data for the button selected shows up
buttons= []
for i in range(0, len(sectors)):
    visible = [False]*len(sectors)
    visible[i] = True
    buttons.append(dict(label = sectors[i][0:30] + "...", method = 'update', args = [{'visible': visible}]))

#updating the menu so it has the buttoms      
fig.update_layout(updatemenus=list([dict(buttons = buttons)]))

#Basic formatting
fig.update_layout(width=1100, height=500,
                  title_text = "Number of Business Owners by Year",
                  xaxis_title="Year",
                  yaxis_title="Number of Business Owners",
                  showlegend=False)

#Making custom x-axis values
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [2002, 2007, 2012],
    )
)

fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='black')

fig.show()

#### Next we we will look at the breakdown of total businesses by sector.

In [ ]:
years = bysector_combined['YEAR'].unique()
bysector_combined = bysector_combined.sort_values(by=['YEAR', 'NAICS_MEANING'])

fig = go.Figure()

for year in years:
    fig.add_trace(
        go.Pie(labels=bysector_combined[bysector_combined['YEAR']==year]['NAICS_MEANING'], 
               values=bysector_combined[bysector_combined['YEAR']==year]['OWNALL'], textinfo='label+percent',
               hole=.3)
    )

#logic for the buttons in the dropdown menus. Only the data for the button selected shows up   
buttons= []
for i in range(0, len(years)):
    visible = [False]*len(years)
    visible[i] = True
    buttons.append(dict(label = str(years[i]) , method = 'update', args = [{'visible': visible}]))

#updating the menu so it has the buttoms      
fig.update_layout(updatemenus=list([dict(buttons = buttons)]))

fig.update_traces(textposition='inside', textinfo='percent')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.update_layout(width=1100, height=500,
                  title_text = "Percent of Business Owners by Sector"
                  )

fig.show()

In [ ]:
#function to create a custom color palette of a given length. I didn't end up using this. 
def create_custom_palette(seaborn_scale, length):
    rgb_colors = sns.color_palette(seaborn_scale, length)
    hex_colors = [matplotlib.colors.to_hex(rgb) for rgb in rgb_colors]
    return hex_colors

print(create_custom_palette('rocket', 20))
    

#### And finally we will look at the trends for each sector and how the different for minorities and non-minorities

In [ ]:
sectors = list(byrace_combined['NAICS_MEANING'].unique())

#adds a plot for the total economic damages, regardless of event 

fig = go.Figure(layout = go.Layout(
    plot_bgcolor='rgba(0,0,0,0)'
))

#loops through the the sectors and shows the trend in minority and non-minority business owners. 
for sector in sectors:

    fig.add_trace(go.Scatter(x=pd.to_numeric(byrace_combined[(byrace_combined['NAICS_MEANING'] == sector) & (byrace_combined['CBOGROUP_MEANING'] == "Minority")]['YEAR']),
                             y=pd.to_numeric(byrace_combined[(byrace_combined['NAICS_MEANING'] == sector) & (byrace_combined['CBOGROUP_MEANING'] == "Minority")]['OWNALL']), 
                             mode="lines+markers", name='Minority', visible=False,
                             marker=dict(
                                size=16,
                                color='red',
                                showscale=False
                             ),
                             line=dict(color='red', width=4),
))

    
    fig.add_trace(go.Scatter(x=pd.to_numeric(byrace_combined[(byrace_combined['NAICS_MEANING'] == sector) & (byrace_combined['CBOGROUP_MEANING'] == "Nonminority")]['YEAR']),
                             y=pd.to_numeric(byrace_combined[(byrace_combined['NAICS_MEANING'] == sector) & (byrace_combined['CBOGROUP_MEANING'] == "Nonminority")]['OWNALL']), 
                             mode="lines+markers", name="Non-minority", visible=False,
                             marker=dict(
                                size=16,
                                color='green', #set color equal to a variable# one of plotly colorscales
                                showscale=False
                             ),
                             line=dict(color='green', width=4),
))

    
#This creates the dropdown menu and makes only one pair of traces visible at a time 
buttons = []
for i in range(0, 2*len(sectors), 2):
    visible = [False]*(2*len(sectors))
    visible[i] = True
    visible[i+1] = True
    buttons.append(dict(label = sectors[int(i/2)][0:25]+ '...', method = 'update', args = [{'visible': visible}]))
    
fig.update_layout(updatemenus=list([dict(buttons = buttons)]))


fig.update_layout(height=500, 
                  title_text = "Number of Minority and Non-minority Owners by Year",
                  xaxis_title="Year",
                  yaxis_title="Number of Business Owners",
                  showlegend=True)

fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [2002, 2007, 2012],
    )
)

fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='black')

fig.show()

### Which states havethe highest levels of minority business owenership

In [ ]:
#grouping the data by state and by the race/ethnicity column
group_analysis_data = analysis_df[['STATE', 'RACE ETH', 'SECTOR']]
races_grouped = group_analysis_data.groupby(['STATE', 'RACE ETH'], as_index=False).count()

#calculating the percentage total for each state
state_totals = races_grouped.groupby('STATE', as_index=False).sum()
races_grouped = races_grouped.merge(state_totals, on='STATE')
races_grouped.columns = ['STATE', 'RACE ETH', 'Owners', 'State Total']
races_grouped['Percent of State Total'] = (races_grouped['Owners'] / races_grouped['State Total'])* 100

fig = go.Figure()

groups = races_grouped['RACE ETH'].unique()

#plots for each race/ethnicity group
for group in groups:
    fig.add_trace(go.Choropleth(
        locations=races_grouped[races_grouped['RACE ETH'] == group]['STATE'], # Spatial coordinates
        z = races_grouped[races_grouped['RACE ETH'] == group]['Percent of State Total'], # Data to be color-coded
        locationmode = 'USA-states', # set of locations match entries in `locations`
        colorscale = 'temps',
        colorbar_title = "Percent",
    ))

#creating the buttons for the dropdown menu    
buttons= []
for i in range(0, len(groups)):
    visible = [False]*len(groups)
    visible[i] = True
    buttons.append(dict(label = str(groups[i]) , method = 'update', args = [{'visible': visible}]))

#updating the menu so it has the buttoms      
fig.update_layout(updatemenus=list([dict(buttons = buttons)]))
fig.update_layout(
    title_text = 'Percent of Business Owners by State',
    geo_scope='usa', # limite map scope to USA
)

fig.show()


## 4) Clustering business ownerships by business characteristics and owner characteristics

In this section, I want to explore creating clusters for different types of businesses based on traits like employment, payroll, receivables, whether the business was family owned, and so on. I then wanted to determine the breakdown of different characteristics of business owners by cluster. 

For this problem I decided to use K-Protoypes Clustering. This algorithm is a mixture of K-Means and K-Modes, and fits this problem well because it can cluster based on both numerical and categorical data. A downside of this algorithm is the computational complexity as you increase the number of data. Without a gpu and hours/days to train the model, I was not able to cluster based on all 1.5 million datapoints. Instead, I had to take random samples of the data. The most I could train with given the limitations of my hardware was 10,000, even after I split the work between all 6 of the processor cores on my laptop.

In [ ]:
#Setting the Cloumns we want to cluster by
cluster_vars = ['EMPLOYMENT_NOISY', 'PAYROLL_NOISY', 'RECEIPTS_NOISY','FEDERAL','STATELOCAL',
                'INDIVIDUALS', 'FAMILYBUS', 'HOMEBASED', 'FRANCHISE']
cluster_df = analysis_df[cluster_vars]
cluster_df = cluster_df.dropna(subset=cluster_vars)

#Standardizing numerical data
cluster_df['EMPLOYMENT_NOISY'] = preprocessing.scale(cluster_df['EMPLOYMENT_NOISY'])
cluster_df['PAYROLL_NOISY'] = preprocessing.scale(cluster_df['PAYROLL_NOISY'])
cluster_df['RECEIPTS_NOISY'] = preprocessing.scale(cluster_df['RECEIPTS_NOISY'])
fit_sample = cluster_df.sample(n=10000, random_state=1)


#Runs the K Prototypes algorithm with 1 through 6 clusters. 
start = time.time()
cost = []
for i in range(1,7):
    model = KPrototypes(n_clusters=i, init='Huang', n_jobs=6, random_state=1) #I split the work between the 6 processor cores on 
    #my laptop
    model.fit(fit_sample, categorical=[3,4,5,6,7,8]) #we specify which columns are categorical
    cost.append(model.cost_)
    
end=time.time()
print("Time to Run: ", end - start, "seconds.")

fig = plt.figure()
#Plots the cost vs. the number of clusters. The lower the cost, the better job it does clustering
plt.plot(range(1,7), cost)
fig.suptitle('Cost vs. Clusters')
plt.xlabel('Clusters')
plt.ylabel('Cost')

In [ ]:
#The model stops being able to initialize well clusters after 5, so we will use 5 clusters.
#I used random states so I wouldn't get new outcomes each time I ran it.
final_model = KPrototypes(n_clusters=5, init='Huang', n_jobs=6, random_state=2)
final_model.fit(cluster_df.sample(n=10000, random_state=2), categorical=[3,4,5,6,7,8])

results = final_model.predict(cluster_df, categorical=[3,4,5,6,7,8])
print(results)

In [ ]:
analysis_df = analysis_df.dropna(subset=cluster_vars)
cluster_column = results
analysis_df['Cluster'] = cluster_column
analysis_df['Cluster'].value_counts()

In [ ]:
print(final_model.cluster_centroids_)

### What do these clusters mean?

#### Cluster 0:
Number of businesses: 449927, Traits: Far below average employees, payroll and receivables; Most often is home-based, but not family owned or a franchise

#### Cluster 1: 
Number of businesses: 572374, Traits: Slightly below average employees, payroll and receivables; More often is not family-owned, home-based, or a franchise. 

#### Cluster 2: 
Number of businesses: 20747, Traits: Slightly above average employees, payroll and receivables; More often is not family-owned, home-based, or a franchise

#### Cluster 3:
Number of businesses: 80, Traits: Far above average employees and receivables with slightly above average payroll; Most often is not family-owned, home-based, or a franchise.

#### Cluster 4:
Number of businesses: 1683, Traits: High above average employees, payroll and receivables; More often is not family-owned, home-based, or a franchise. 


In [ ]:
analysis_df.head()

In [ ]:
#To get an idea about the clusters, we can plot some data
plot_df = analysis_df[(analysis_df['PAYROLL_NOISY'] < 1000000) & (analysis_df['RECEIPTS_NOISY'] < 8000000)]
#transforming the data with the natural logarithm so it is less skewed.
plot_df['log_payroll'] = np.log2(plot_df['PAYROLL_NOISY'])
plot_df['log_receipts'] = np.log2(plot_df['RECEIPTS_NOISY'])
fig = px.scatter(plot_df, x="log_payroll", y="log_receipts", color="Cluster")
fig.update_layout(title='Company Payroll vs. Receipts (Vizualisation of Clusters)')
fig.show()

In [ ]:
def cluster_percentages(df, column):
    """Function to determine the percentage of each unique value in the 'column' for each of the 5 clusters"""
    clusters = [0,1,2,3,4]
    
    #finding the percentage breakdown for the total data set, regardless of cluter. 
    final = df[column].value_counts(normalize=True).reset_index()
    final.columns = [column, 'Percent']
    final['Cluster'] = 'Total'
    
    #creating a dataframe for the percentages of each cluster
    for i in clusters:
        cluster_df = df[df['Cluster'] == i]
        cluster_perc = cluster_df[column].value_counts(normalize=True).reset_index()
        cluster_perc.columns = [column, 'Percent']
        cluster_perc['Cluster'] = "Cluster " + str(i)
        
        final = pd.concat([final, cluster_perc])
    
    final['Percent'] = final['Percent'] * 100
    final[column] = final[column].astype(str)
        
    return final



def graph_percentages(df, column):
    """Plots the percentage of the values in the given column present in each cluster"""
    dataframe = cluster_percentages(df, column)
    dataframe = dataframe.sort_values(by='Percent')
    fig = go.Figure()
    values = df[column].unique()
    
    #plots bar charts for each unique value in the column based on the percentage breakdown by cluster.
    for v in values:
        
        #changes only the 'Total' values to red bars. 
        cluster_list = dataframe[dataframe[column] == v]['Cluster'].to_list()
        colors = ['Blue'] * len(cluster_list)
        colors_final = ['Red' if i == "Total" else "Blue" for i in cluster_list]
        
        fig.add_trace(
            go.Bar(x = dataframe[dataframe[column] == v]['Cluster'],
                      y = dataframe[dataframe[column] == v]['Percent'],
                      name=v,
                      visible=False,
                      marker_color=colors_final))
    
    #allows selections based on all of the values in the given column
    buttons = []
    for i in range(0, len(values)):
        visible = [False]*len(values)
        visible[i] = True
        buttons.append(dict(label = values[i], method = 'update', args = [{'visible': visible}])) 
    
    fig.update_layout(updatemenus=list([dict(buttons = buttons)]))
    #formatting
    fig.update_layout(height=500, 
                  title_text = "Percent of Owners in Each Cluster",
                  xaxis_title="Clusters",
                  yaxis_title="Percent",
                  showlegend=False)
    
    fig.show()

### Minority

In [ ]:
graph_percentages(analysis_df, 'Minority')

### Race/Ethnicity

In [ ]:
graph_percentages(analysis_df, 'RACE ETH')

### Born in the United States

In [ ]:
graph_percentages(analysis_df, 'BORNUS1')

## Conclusion

The purpose of this project was to paint broad strokes about patterns that arise in business ownership in the United States. While we were interested in general trends, like the how the number of business owners were changing in certain sectors, the main goal of was to identify patterns in the demographics of business owners. 

In terms of the trend from 2002 to 2012, we observed many different combination of outcomes. For some sectors, like information and technology, both minority and non-minority ownership was increasing. In others, like real estate, both minority and nonminority ownership was increasing, but nonminority ownership was increasing much faster. Yet another outcome was the instance of minority ownership increasing and non-minority ownership decreasing.

We also observed trends in geographic trends in the data. One takeaway was that even in states with high minority populations, one minority would tend to have a much larger presence in business ownership than the others. This points to the observation that while the United States is anecdotally considered a "melting pot", business ownership tends to stratify rather than mix together. 

Another important finding was was the demographic distribution amongst the clusters different types of businesses. While minorities make up about 18 percent of all business owners, they make up only about 5.6 percent of the owners of businesses belonging to cluster 4 (generally larger companies). Whether this discrepancy is a result of certain societal blockades towards minorities owning these more "successful" companies cannot be answered by this dataset, but the discrepancy is certainly there.  

While I consider the research into this topic successful, there are definitely ways to improve. For one, I didn't have the hardware to take full advantage of the K-prototypes clustering algorithm. My ability to cluster was limited to random samples of data and all feature selection was done on a trial and error basis.
Additionally, data on this topic is not as readily available as I believe it should be. As a result of this. I was limited to using data from 2007 to create clusters, as this was the only sizeable dataset available to the public. 

### Zenodo Publication: https://zenodo.org/record/4321611#.X9e8CBNKhhE
### Gitbub Publication: https://github.com/nickblackmore/DATS-6103-Individual-Project-3-Nicholas-Blackmore
### Github.io Publication: https://nickblackmore.github.io/DATS-6103-Individual-Project-3-Nicholas-Blackmore-Web-Page